In [1]:
from mnemonic import Mnemonic
from fastecdsa import keys, curve
import binascii
import hashlib
import hmac



In [2]:
"""here i create a class to store data in a tree like structure"""

class TreeNode():
    
    def __init__(self,parent):
        self.parent=parent
        self.pvt_key=None
        self.pub_key=None
        self.chain_code =None
        self.children = []
    

In [3]:
"""Here I define the function needed to generate the seed which is going to be used later """

def generate_seed(language,passphrase,strenght):
   mnemo = Mnemonic(language)
   words = mnemo.generate(strength=strenght)
   seed = mnemo.to_seed(words, passphrase=passphrase)
   print("Entropy=" + str( binascii.hexlify(mnemo.to_entropy(words)))) #prints the entropy in hex
   print("Words= " + words)   
   print("Seed = "+str(binascii.hexlify(seed))) #prints the seed in hex 
   return seed
   


In [4]:
seed=generate_seed("english","test",128)

Entropy=b'83ef384efdb87bfcc6060fbd45c7f458'
Words= loop keep beauty window marble wrist blossom sea run comfort wonder rain
Seed = b'ba03f39d99bccf30c87bd0e76c7b70f13e260927cce2daa39f3234da3f69e34a0daadef5c75557d7b91d44e5ce2fbfcbe0969ccb1de111a32a30227a31d5400a'


In [5]:
"""In this section I derive from the seed the master private key and the master chain code, the private key is the one used to have control over your bank account represented by the public key. The bitcoin address is generated/corresponds from the public key"""

def get_master_info(seed):
    master=TreeNode(None)
    master_hmac= hmac.new(seed,None,hashlib.sha512).hexdigest()
    master_pvt_key=master_hmac[:64]
    master.pvt_key=master_pvt_key
    master_chain_code=master_hmac[64:]
    master.chain_code=master_chain_code
    pvt=int(master_pvt_key, 16)
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1)
    pub_key= hex(master_pub_key.y)[2:4]+hex(master_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
    master.pub_key=pub_key
    return master


In [6]:
master_info=get_master_info(seed)
master_info.pub_key


'fe1279b9396db996fe17e55113b3b3f9a020282801ea8e1352675a60dd96a30f8e'

In [7]:
"""In this section I define a function to derive child addresses using hardened derivation"""

def create_hardened_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pvt_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)
            

In [8]:
"""in this section I create a children from the public key of the master"""

def create_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pub_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)

In [9]:
create_children(master_info,"00000000",2)
create_hardened_children(master_info,"80000000",1)

for i in master_info.children:
        print("Private Key " + i.pvt_key)
        print("Public Key " + i.pub_key)
        print("Chain Code " + i.chain_code)
        print("-------------------------------------------------------------------------")
        

Private Key 7c91c2d53d49ad6536891a40b9bb7d43f0105e6649071655568a3ad80ffbc1b8
Public Key 87e4d962c7f7c69769681b80a21ba9c709a6ab8ea4f8a115ca05d4ca9f29dca4f6
Chain Code e5fca57064f1460fa5f4667798e955795ead6b21a1a868134453ee6e5ea5d9f2
-------------------------------------------------------------------------
Private Key 052b3eab86ded2803f372b3112fe00f248501e37183165ae6f9ebee8e038e395
Public Key f679b138182a444220839152642b35cf8a650117e458ad5e9c7c1f5202e1c1627c
Chain Code de7fa24841fb0e521434b8765497cba248a9d4fd1da62ff4138847122e797af4
-------------------------------------------------------------------------
Private Key bed661f7560017f5fdd0816c09028ddaa27c38fc00b70193f1821a2706b7a62a
Public Key 57d149bbc1a2035a34fc916740fa9829534b21a6b10f1d563e9133a45171333b11
Chain Code e992093173a630b4d4e54fdd962b463ee5da61d6ae73f359c31c258959e62c9c
-------------------------------------------------------------------------


In [10]:
"""Lets try to create some grandkids"""

for i in master_info.children:
    create_hardened_children(i,"80000000",1)
    create_children(i,"00000000",2)
    for j in i.children:
        print("Private Key " + j.pvt_key)
        print("Public Key " + j.pub_key)
        print("Chain Code " + j.chain_code)
        print("-------------------------------------------------------------------------")
        




Private Key 5bb4d5a17163d354318c5cf1c976e1d88959845956727cf6673f7bcda9929862
Public Key 6b4b7d53ed6fcefa38e4b70baa777483a2b11df3a003b5fb3c820ead5f1ee866a8
Chain Code 95ba24bf27ea062619eef67eba7684c01d2172cfa787e4937d92d9931e084555
-------------------------------------------------------------------------
Private Key 27a695763b39acf25885515dabafa9685f1d51dbe2459c2540da76d1a543d8ef
Public Key c03332b9e66916e278d969925f763a704baa914f0448860cd488f852190e83284
Chain Code bf49cbe3b185641657e2cbb031422995b7667bf7e46aac06bf558d2bb5af4c51
-------------------------------------------------------------------------
Private Key bcd110c5236aabb35939691082db5bf1dcaef9e1adee8fab355ca4043463d940
Public Key ae222bf6c64e53be1879fb04dfc7f92f5d3d7f226122912729004ce47062d8bb7e
Chain Code eee2af2d64f5ae78b0a8965ebc74a7adf63768368587b6a7a35504bc6397c441
-------------------------------------------------------------------------
Private Key 6c601ff1d32b04ec3695698695ac25d49a544f05391a5ec548678419af87fa59
Public Ke